<a href="https://colab.research.google.com/github/gonzalorodriguezrovira/TFG_SkinAnalizer/blob/main/Codigo/TFG_Calculo_ITA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Modulos**

In [ ]:
# Carga de modulos
import math
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import PIL
import csv
import cv2

from google.colab import auth
from skimage import io, color
from google.colab import drive

# Preprocesamiento y entrenamiento de TensorFlow automático
AUTO = tf.data.AUTOTUNE

# Para extraer ITA es indiferente, por lo que usamos un tamaño pequeño
MODEL_RESOLUTION = "256x256"
TFG_METADATA="gs://tfgtfrecords/256x256/train*.tfrec"

# **Funciones para carga de datasets**

In [ ]:
# Carga dataset de kaggle
def load_GCS():
    auth.authenticate_user()
    drive.mount('/content/gdrive', force_remount=True)

    dataset_files = {}

    dataset_files[MODEL_RESOLUTION] = tf.io.gfile.glob(TFG_METADATA)

    dataset = {}
    for resolution, files in dataset_files.items():
        dataset[MODEL_RESOLUTION] = load_dataset(files)

    df_ISIC=pd.read_csv('/content/gdrive/MyDrive/df_ISIC_MOD.csv')

    return df_ISIC,dataset

# Prepara la imagen redimensionandola y normalizando los pixeles
def prepare_image(image_data):
  image = tf.image.decode_jpeg(image_data, channels=3)
  image = tf.reshape(image, [256,256, 3])
  image = tf.cast(image, tf.float32) / 255.0
  return image

# Función para extraer la informacion de los TFRecord
def read_labeled_tfrecord_with_file_name(example):
    LABELED_TFREC_FORMAT = {
        "image": tf.io.FixedLenFeature([], tf.string),
        "image_name": tf.io.FixedLenFeature([], tf.string),
        "patient_id": tf.io.FixedLenFeature([], tf.int64),
        "sex": tf.io.FixedLenFeature([], tf.int64),
        "age_approx": tf.io.FixedLenFeature([], tf.int64),
        "anatom_site_general_challenge": tf.io.FixedLenFeature([], tf.int64),
        "diagnosis": tf.io.FixedLenFeature([], tf.int64),
        "target": tf.io.FixedLenFeature([], tf.int64),
    }
    example = tf.io.parse_single_example(example, LABELED_TFREC_FORMAT)
    image = prepare_image(example['image'])
    image_name = example['image_name']
    diagnosis = tf.cast(example['diagnosis'], tf.int32)
    return image, image_name, diagnosis


# Filtro para eliminar las imagenes que contienen dowsampled
def filter_images(image, image_name, diagnosis):
  return not tf.strings.regex_full_match(image_name,"^((.*_downsampled))")

# Función para cargar en un dataset los TFRecords
# Devuelve un dataset con el formato (image, image_name, diagnosis)
def load_dataset(filenames):
    dataset = tf.data.TFRecordDataset(filenames, num_parallel_reads=AUTO)
    dataset = dataset.map(read_labeled_tfrecord_with_file_name,
                          num_parallel_calls=AUTO).filter(filter_images)
    return dataset


# **Funciones para calculo de ITA**

In [ ]:
# Detecting Melanoma Fairly: Skin Tone Detection and Debiasing
# for Skin Lesion Classification
# https://github.com/pbevan1/Detecting-Melanoma-Fairly/blob/main/preprocessing.py
# Hair removal for ITA calculation
def hair_remove(image):
    # Convert image to grayScale
    grayScale = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    # Kernel for morphologyEx
    kernel = cv2.getStructuringElement(1, (17, 17))
    # Apply MORPH_BLACKHAT to grayScale image
    blackhat = cv2.morphologyEx(grayScale, cv2.MORPH_BLACKHAT, kernel)
    # Apply thresholding to blackhat
    _, threshold = cv2.threshold(blackhat, 10, 255, cv2.THRESH_BINARY)
    # Inpaint with original image and threshold image
    final_image = cv2.inpaint(image, threshold, 1, cv2.INPAINT_TELEA)
    return final_image

# Calculates Fitzpatrick skin type of an image using Kinyanjui et al.'s thresholds
def get_sample_ita_kin(img):
    ita_bnd_kin = -1
    try:
        rgb = img
        rgb = hair_remove(rgb)
        lab = color.rgb2lab(rgb)
        ita_lst = []
        ita_bnd_lst = []

        # Taking samples from different parts of the image
        L1 = lab[230:250, 115:135, 0].mean()
        b1 = lab[230:250, 115:135, 2].mean()

        L2 = lab[5:25, 115:135, 0].mean()
        b2 = lab[5:25, 115:135, 2].mean()

        L3 = lab[115:135, 5:25, 0].mean()
        b3 = lab[115:135, 5:25, 2].mean()

        L4 = lab[115:135, 230:250, 0].mean()
        b4 = lab[115:135, 230:250, 2].mean()

        L5 = lab[216:236, 216:236, 0].mean()
        b5 = lab[216:236, 216:236, 2].mean()

        L6 = lab[216:236, 20:40, 0].mean()
        b6 = lab[216:236, 20:40, 2].mean()

        L7 = lab[20:40, 20:40, 0].mean()
        b7 = lab[20:40, 20:40, 2].mean()

        L8 = lab[20:40, 216:236, 0].mean()
        b8 = lab[20:40, 216:236, 2].mean()

        L_lst = [L1, L2, L3, L4, L5, L6, L7, L8]
        b_lst = [b1, b2, b3, b4, b5, b6, b7, b8]

        # Calculating ITA values
        for L, b in zip(L_lst, b_lst):
            ita = math.atan((L - 50) / b) * (180 / math.pi)
            ita_lst.append(ita)

        # Using max ITA value (lightest)
        ita_max = max(ita_lst)

        # Getting skin shade band from ITA
        if ita_max > 55:
            ita_bnd_kin = 1
        if 41 < ita_max <= 55:
            ita_bnd_kin = 2
        if 28 < ita_max <= 41:
            ita_bnd_kin = 3
        if 19 < ita_max <= 28:
            ita_bnd_kin = 4
        if 10 < ita_max <= 19:
            ita_bnd_kin = 5
        if ita_max <= 10:
            ita_bnd_kin = 6
    except Exception:
        pass

    return ita_bnd_kin

# **Calculo ITA y almacenamiento**

In [ ]:
# Se carga los metadatos y el dataset
df_ISIC,dataset=load_GCS()

# Se almacena en drive en caso de que aun no lo este
if not os.path.exists('/content/gdrive/MyDrive/df_ITA.csv'):
  df_ITA=[]
  df_ITA.to_csv('/content/drive/MyDrive/df_ITA.csv', index=False)
  header = ['image_name', 'ITA']
  with open('/content/gdrive/MyDrive/df_ITA.csv', 'w', encoding='UTF8') as f:
    writer = csv.writer(f)
    writer.writerow(header)
    idx = 0
    for img, file, target in iter(dataset[MODEL_RESOLUTION]):
      image = np.clip(img.numpy() * 255, 0, 255).astype(np.uint8)
      image = hair_remove(image)
      ITA = get_sample_ita_kin(image)
      writer.writerow([file.numpy().decode('ascii'), ITA])
      idx = idx + 1

Mounted at /content/gdrive


In [ ]:
# Se comprueba que se almacene correctamente
df_ITA=pd.read_csv('/content/gdrive/MyDrive/df_ITA.csv')
df_ITA

,image_name,ITA
0,ISIC_0000002,6
1,ISIC_0000006,6
2,ISIC_0000074,6
3,ISIC_0000226,1
4,ISIC_0000234,1
...,...,...
23213,ISIC_0073086,6
23214,ISIC_0073100,6
23215,ISIC_0073104,1
23216,ISIC_0073235,1


Este notebook fue inspirado por:

Detecting Melanoma Fairly: Skin Tone Detection and Debiasing for Skin Lesion Classification
https://github.com/pbevan1/Detecting-Melanoma-Fairly/blob/main/preprocessing.py